In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
#in here are automatic intervention counts that may be wrong and do not exclude intersections and traffic
results = """
Camera v1 (May)
{'mae': 0.23088668332126538, 'rmse': 0.30444885103275254, 'max': 1.3838765496927377, 'failure_rate': 0.961781827385472,'distance': 8464.334414890189, 'distance_per_intervention': 2116.083603722547, 'interventions': 2, 'whiteness': 24.62509993931149, 'cmd_whiteness': 65.02619290238128, 'expert_whiteness': 23.387318346549534}
Camera v2 (May)
{'mae': 0.23823662499297601, 'rmse': 0.3130526038207709, 'max': 1.2721541136976486, 'failure_rate': 0.6917558642678205,'distance': 8363.17488650764, 'distance_per_intervention': 1045.396860813455, 'interventions': 4, 'whiteness': 33.46032825076525, 'cmd_whiteness': 107.73348095698013, 'expert_whiteness': 23.387318346549534}
Camera v3 (May)
{'mae': 0.2402807594393857, 'rmse': 0.3086952556841057, 'max': 1.1453249999323865, 'failure_rate': 0.6642254960401941,'distance': 8389.87918022878, 'distance_per_intervention': 932.2087978031977, 'interventions': 7, 'whiteness': 29.698904637469028, 'cmd_whiteness': 67.55371029928838, 'expert_whiteness': 23.387318346549534}
LiDAR v1 (Nov)
{'mae': 0.2164273510701372, 'rmse': 0.28794647831162357, 'max': 1.4814049216935727, 'failure_rate': 0.4200621692010418,'distance': 8442.519560651908, 'distance_per_intervention': 2110.629890162977, 'interventions': 2, 'whiteness': 22.95757235057836, 'cmd_whiteness': 60.71982088365033, 'expert_whiteness': 22.923865138522743}
LiDAR v2 (Nov)
{'mae': 0.23951848356004346, 'rmse': 0.32021653474413686, 'max': 1.410733995029802, 'failure_rate': 0.9813789632611978,'distance': 8465.916553584271, 'distance_per_intervention': 2116.479138396068, 'interventions': 2, 'whiteness': 17.707242631777568, 'cmd_whiteness': 40.788217567179025, 'expert_whiteness': 22.923865138522743}
LiDAR v3 (Nov)
{'mae': 0.2536249345345471, 'rmse': 0.3528786544085774, 'max': 1.6002952563202872, 'failure_rate': 2.1795842800639567, 'distance': 8432.327240726016, 'distance_per_intervention': 1686.465448145203, 'interventions': 3, 'whiteness': 18.841146974682626, 'cmd_whiteness': 56.74452412521819, 'expert_whiteness': 22.923865138522743}
Camera overfit (May)
{'mae': 0.24525640421044248, 'rmse': 0.3291476438504756, 'max': 1.2536699163819658, 'failure_rate': 1.3851891315929674,'distance': 8489.138533056424, 'distance_per_intervention': 2829.7128443521415, 'interventions': 3, 'whiteness': 23.0742793721048, 'cmd_whiteness': 65.94352651549603, 'expert_whiteness': 23.387318346549534}
LiDAR overfit (Nov)
{'mae': 0.2598746390647679, 'rmse': 0.3802804511875996, 'max': 1.6692438325569796, 'failure_rate': 4.375209942895532,'distance': 8436.864018329401, 'distance_per_intervention': 2812.2880061098003, 'interventions': 0, 'whiteness': 19.209361200653984, 'cmd_whiteness': 38.54266349696802, 'expert_whiteness': 22.923865138522743}
LiDAR night
{'mae': 0.23909138446324246, 'rmse': 0.32493206282364406, 'max': 1.659170327718653, 'failure_rate': 1.2672413793103448,'distance': 8216.289369532735, 'distance_per_intervention': 684.6907807943945, 'interventions': 8, 'whiteness': 25.568653921414345, 'cmd_whiteness': 69.0679117344516, 'expert_whiteness': 22.923865138522743}
LiDAR night #2
{'mae': 0.23140690498687244, 'rmse': 0.3148313254633656, 'max': 1.6218812651299235, 'failure_rate': 1.5501905972045744,'distance': 8376.642020810223, 'distance_per_intervention': 1396.1070034683705, 'interventions': 3, 'whiteness': 20.270503405325538, 'cmd_whiteness': 49.14729491131972, 'expert_whiteness': 22.923865138522743}
LiDAR overfit night
{'mae': 0.25314860436451386, 'rmse': 0.3459086938928751, 'max': 1.630496452339, 'failure_rate': 1.5185648727576135,'distance': 8521.477898587687, 'distance_per_intervention': 2840.4926328625625, 'interventions': 1, 'whiteness': 21.52354723505918, 'cmd_whiteness': 67.14565934366986, 'expert_whiteness': 22.923865138522743}
LiDAR Winter v1
{'mae': 0.24077287766041175, 'rmse': 0.32074558957088006, 'max': 1.5369283242890857, 'failure_rate': 0.9426619389431808,'distance': 8080.505297855708, 'distance_per_intervention': 404.0252648927854, 'interventions': 19, 'whiteness': 38.38490904517742, 'cmd_whiteness': 107.12677614094522, 'expert_whiteness': 22.923865138522743}
LiDAR Winter v2
{'mae': 0.2971095664175751, 'rmse': 0.4105259810386456, 'max': 1.5934707554255736, 'failure_rate': 3.536532641591007,'distance': 7980.378562030141, 'distance_per_intervention': 306.93763700115926, 'interventions': 22, 'whiteness': 37.597063418165874, 'cmd_whiteness': 118.53725563410337, 'expert_whiteness': 22.923865138522743}
LiDAR Winter v3
{'mae': 0.256597960271629, 'rmse': 0.3449099860369303, 'max': 1.6163590616877834, 'failure_rate': 1.643884570713839,'distance': 7698.891989550286, 'distance_per_intervention': 197.4074869115458, 'interventions': 34, 'whiteness': 42.15369836835372, 'cmd_whiteness': 145.37316860331046, 'expert_whiteness': 22.923865138522743}
LiDAR v2 day 2 (Nov)
{'mae': 0.21934381266983838, 'rmse': 0.31392314409302075, 'max': 1.6426648453005086, 'failure_rate': 1.968503937007874,'distance': 8491.618679568586, 'distance_per_intervention': 2830.5395598561954, 'interventions': 0, 'whiteness': 19.17217743456574, 'cmd_whiteness': 33.89063165134626, 'expert_whiteness': 22.923865138522743}
LiDAR intensity (Nov)
{'mae': 0.3267694308573402, 'rmse': 0.4732339915443102, 'max': 1.6593815307920154, 'failure_rate': 7.01828292189738,'distance': 8446.16604232979, 'distance_per_intervention': 2111.5415105824477, 'interventions': 2, 'whiteness': 23.961064265768467, 'cmd_whiteness': 53.783987250236294, 'expert_whiteness': 22.923865138522743}
LiDAR range (Nov)
{'mae': 0.6122139347238139, 'rmse': 0.7695435033217324, 'max': 1.6757036288168157, 'failure_rate': 19.954562665656947,'distance': 1679.012717905246, 'distance_per_intervention': 76.31875990478392, 'interventions': 22, 'whiteness': 29.858233482499045, 'cmd_whiteness': 64.02717916493413, 'expert_whiteness': 22.923865138522743}
LiDAR ambience (Nov)
{'mae': 0.728352878106602, 'rmse': 0.8773714729402442, 'max': 1.7017581466320282, 'failure_rate': 17.49271137026239,'distance': 329.4733138441798, 'distance_per_intervention': 17.340700728641043, 'interventions': 19, 'whiteness': 168.18447945397463, 'cmd_whiteness': 555.1688851900144, 'expert_whiteness': 22.923865138522743}

"""

In [3]:
results = results.split('\n')[1:-1]
names = results[::2]
experiment_results = results[1::2]

In [4]:
import warnings
warnings.filterwarnings('ignore')

metrics_df = pd.DataFrame(columns=["model", 'mae', 'rmse', 'max', 'failure_rate', 'distance', 'distance_per_intervention', 'interventions', 'whiteness', 'cmd_whiteness', 'expert_whiteness'])
for name, res in zip(names, experiment_results):
    print(name)
    res = json.loads(res.replace("'", '"'))
    metrics = pd.DataFrame(data=res, index=[0])
    metrics['model'] = name
    metrics_df = metrics_df.append(metrics, ignore_index=True)
metrics_df.set_index("model");
metrics_df = metrics_df[["model", 'mae', 'rmse', 'max', 'failure_rate', 'distance', 'distance_per_intervention', 'interventions', 'whiteness', 'cmd_whiteness', 'expert_whiteness']]
metrics_df

Camera v1 (May)
Camera v2 (May)
Camera v3 (May)
LiDAR v1 (Nov)
LiDAR v2 (Nov)
LiDAR v3 (Nov)
Camera overfit (May)
LiDAR overfit (Nov)
LiDAR night
LiDAR night #2
LiDAR overfit night
LiDAR Winter v1
LiDAR Winter v2
LiDAR Winter v3
LiDAR v2 day 2 (Nov)
LiDAR intensity (Nov)
LiDAR range (Nov)
LiDAR ambience (Nov)


,model,mae,rmse,max,failure_rate,distance,distance_per_intervention,interventions,whiteness,cmd_whiteness,expert_whiteness
0,Camera v1 (May),0.230887,0.304449,1.383877,0.961782,8464.334415,2116.083604,2,24.625100,65.026193,23.387318
1,Camera v2 (May),0.238237,0.313053,1.272154,0.691756,8363.174887,1045.396861,4,33.460328,107.733481,23.387318
2,Camera v3 (May),0.240281,0.308695,1.145325,0.664225,8389.879180,932.208798,7,29.698905,67.553710,23.387318
3,LiDAR v1 (Nov),0.216427,0.287946,1.481405,0.420062,8442.519561,2110.629890,2,22.957572,60.719821,22.923865
4,LiDAR v2 (Nov),0.239518,0.320217,1.410734,0.981379,8465.916554,2116.479138,2,17.707243,40.788218,22.923865
5,LiDAR v3 (Nov),0.253625,0.352879,1.600295,2.179584,8432.327241,1686.465448,3,18.841147,56.744524,22.923865
6,Camera overfit (May),0.245256,0.329148,1.253670,1.385189,8489.138533,2829.712844,3,23.074279,65.943527,23.387318
7,LiDAR overfit (Nov),0.259875,0.380280,1.669244,4.375210,8436.864018,2812.288006,0,19.209361,38.542663,22.923865
8,LiDAR night,0.239091,0.324932,1.659170,1.267241,8216.289370,684.690781,8,25.568654,69.067912,22.923865
9,LiDAR night #2,0.231407,0.314831,1.621881,1.550191,8376.642021,1396.107003,3,20.270503,49.147295,22.923865


In [5]:
metrics_df.insert(11, 'ol_season_whiteness', [63.201572, 54.70426, 63.663876,
                                              108.563, 97.656, 107.233, 
                                              59.91,81.347, #overfits
                                              107.233, 107.233, 81.3, #night
                                400.582, 462.965, 370.778,
                                97.656, 111.384, 101.421, 204.356])

metrics_df.insert(12, 'ol_mae', [6.2567,6.7220,6.6836,
                                 7.944, 8.223, 8.132, 
                                 5.9565,6.299, #overfits
                                 8.132, 8.132, 6.3, #night
                                 43.140, 58.815, 32.979, #winter
                                 8.223, 7.547, 11.102, 12.570])

In [6]:
metrics_df["distance_per_intervention"] = metrics_df.apply(lambda x: x['distance']/x['interventions'] if x['interventions'] != 0 else x['distance'], axis=1)

In [7]:
metrics_df

,model,mae,rmse,max,failure_rate,distance,distance_per_intervention,interventions,whiteness,cmd_whiteness,expert_whiteness,ol_season_whiteness,ol_mae
0,Camera v1 (May),0.230887,0.304449,1.383877,0.961782,8464.334415,4232.167207,2,24.625100,65.026193,23.387318,63.201572,6.2567
1,Camera v2 (May),0.238237,0.313053,1.272154,0.691756,8363.174887,2090.793722,4,33.460328,107.733481,23.387318,54.704260,6.7220
2,Camera v3 (May),0.240281,0.308695,1.145325,0.664225,8389.879180,1198.554169,7,29.698905,67.553710,23.387318,63.663876,6.6836
3,LiDAR v1 (Nov),0.216427,0.287946,1.481405,0.420062,8442.519561,4221.259780,2,22.957572,60.719821,22.923865,108.563000,7.9440
4,LiDAR v2 (Nov),0.239518,0.320217,1.410734,0.981379,8465.916554,4232.958277,2,17.707243,40.788218,22.923865,97.656000,8.2230
5,LiDAR v3 (Nov),0.253625,0.352879,1.600295,2.179584,8432.327241,2810.775747,3,18.841147,56.744524,22.923865,107.233000,8.1320
6,Camera overfit (May),0.245256,0.329148,1.253670,1.385189,8489.138533,2829.712844,3,23.074279,65.943527,23.387318,59.910000,5.9565
7,LiDAR overfit (Nov),0.259875,0.380280,1.669244,4.375210,8436.864018,8436.864018,0,19.209361,38.542663,22.923865,81.347000,6.2990
8,LiDAR night,0.239091,0.324932,1.659170,1.267241,8216.289370,1027.036171,8,25.568654,69.067912,22.923865,107.233000,8.1320
9,LiDAR night #2,0.231407,0.314831,1.621881,1.550191,8376.642021,2792.214007,3,20.270503,49.147295,22.923865,107.233000,8.1320


In [8]:
metrics_df.rename(columns={'whiteness': 'cl_whiteness'}, inplace=True)
metrics_df.rename(columns={'cmd_whiteness': 'cl_cmd_whiteness'}, inplace=True)
metrics_df.rename(columns={'mae': 'cl_mae'}, inplace=True)

In [9]:
metrics_df = metrics_df.astype({"interventions": float})

In [10]:
#metrics_df.drop([13, 14, 18], inplace=True)

In [11]:
metrics_df.drop(['rmse', 'max', 'expert_whiteness'], axis=1, inplace=True)

In [12]:
cols = metrics_df.columns.tolist()
print(cols)
order=['model', 'distance', 'interventions','distance_per_intervention', 'cl_mae', 'failure_rate', 'cl_whiteness', 'ol_mae','ol_season_whiteness']
metrics_df=metrics_df[order]

['model', 'cl_mae', 'failure_rate', 'distance', 'distance_per_intervention', 'interventions', 'cl_whiteness', 'cl_cmd_whiteness', 'ol_season_whiteness', 'ol_mae']


In [13]:
metrics_df.style.format({
    'cl_mae': "{:.4f}m",
    'rmse': "{:.4f}m",
    'max': "{:.4f}m",
    'failure_rate': "{:.2f}%",
    'interventions': "{:.0f}",
    'cl_whiteness': "{:.2f}°/s",
    'ol_whiteness': "{:.2f}°/s",
    'cl_cmd_whiteness': "{:.2f}°/s",
    'ol_season_whiteness': "{:.2f}°/s",
    'ol_mae': "{:.2f}°",
    'distance': "{:.2f}m",
    'distance_per_intervention': "{:.2f}m"
})

,model,distance,interventions,distance_per_intervention,cl_mae,failure_rate,cl_whiteness,ol_mae,ol_season_whiteness
0,Camera v1 (May),8464.33m,2,4232.17m,0.2309m,0.96%,24.63°/s,6.26°,63.20°/s
1,Camera v2 (May),8363.17m,4,2090.79m,0.2382m,0.69%,33.46°/s,6.72°,54.70°/s
2,Camera v3 (May),8389.88m,7,1198.55m,0.2403m,0.66%,29.70°/s,6.68°,63.66°/s
3,LiDAR v1 (Nov),8442.52m,2,4221.26m,0.2164m,0.42%,22.96°/s,7.94°,108.56°/s
4,LiDAR v2 (Nov),8465.92m,2,4232.96m,0.2395m,0.98%,17.71°/s,8.22°,97.66°/s
5,LiDAR v3 (Nov),8432.33m,3,2810.78m,0.2536m,2.18%,18.84°/s,8.13°,107.23°/s
6,Camera overfit (May),8489.14m,3,2829.71m,0.2453m,1.39%,23.07°/s,5.96°,59.91°/s
7,LiDAR overfit (Nov),8436.86m,0,8436.86m,0.2599m,4.38%,19.21°/s,6.30°,81.35°/s
8,LiDAR night,8216.29m,8,1027.04m,0.2391m,1.27%,25.57°/s,8.13°,107.23°/s
9,LiDAR night #2,8376.64m,3,2792.21m,0.2314m,1.55%,20.27°/s,8.13°,107.23°/s
